In [1]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [3]:
df=pd.read_csv("IMDB dataset.csv")

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.groupby("sentiment").describe()

review                                                            \
           count unique                                                top   
sentiment                                                                    
negative   25000  24698  This show comes up with interesting locations ...   
positive   25000  24884  Loved today's show!!! It was a variety and not...   

                
          freq  
sentiment       
negative     3  
positive     5

In [7]:
df["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
class_names = ['positive', 'negative']

In [38]:
encoding = {
    
    'positive': 1,
    'negative': 0
}

In [39]:
reviews = df['review'].values.tolist()
sentiments = df['sentiment'].tolist()

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=.2)

In [41]:
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [42]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=512, 
                                                                       max_features=51200)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [43]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 512
done.


In [44]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [29]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
6667/6667 [==============================] - 86003s 13s/step - loss: 0.2357 - accuracy: 0.9042 - val_loss: 0.1905 - val_accuracy: 0.9274
Epoch 2/3
6667/6667 [==============================] - 69531s 10s/step - loss: 0.1481 - accuracy: 0.9462 - val_loss: 0.1525 - val_accuracy: 0.9445
Epoch 3/3
6667/6667 [==============================] - 71870s 11s/step - loss: 0.0486 - accuracy: 0.9849 - val_loss: 0.1607 - val_accuracy: 0.9498


In [30]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

    positive       0.95      0.95      0.95      5041
    negative       0.95      0.95      0.95      4959

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000



array([[4794,  247],
       [ 255, 4704]], dtype=int64)

In [45]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['positive', 'negative']

In [47]:
import time 

message = 'Horrable movie. how can you watch this?'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: negative (1.01)


In [48]:
predictor.save("models/bert_model")